Calculate the global mean T2m for 1979-2014 and 2070-2099 of the CMIP6 models.  Outputs both the global mean T2m of the ensemble mean and the first ensemble member.  Using the historical simulations for the past and the SSP5-8.5 simulations for the future  This code works on a CMIP archive that is internal to the Climate and Global Dynamics Laboratory, NCAR.  I you want to run this on your own archive, you'd need to modify histpath and rcp85path and make sure the directory structure is set up in the right way.

In [1]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
from numpy import nan
import sys
import warnings
import math
from glob import glob

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import calendar_utils as cal

importlib.reload(read)
importlib.reload(avg)

warnings.filterwarnings('ignore')

Set path for CMIP6 models (historical and SSP5-8.5) and output directory

In [2]:
histpath="/project/cmip6/historical/Amon/"
ssp585path="/project/cmip6/ssp585/Amon/"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Information on the models being used is provided in cmip6csvinfo.csv.  This contains information on the models, number of member and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [3]:
cmip6models=pd.read_csv('../cmip6csvinfo.csv')

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for Past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

This is the main part of the script.  It loops over models and ensemble members and calculates the global mean T2m for JJA for both the ensemble mean and the first member.  These are then output to netcdf

In [4]:
models=cmip6models['Model']
nmodels=models.size

tas1memp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="Model", name="tas1memp")
tas1memf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="Model", name="tas1memf")
tasemp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="Model", name="tasemp")
tasemf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="Model", name="tasemf")

# loop over models
for index, modname in models.iteritems():
    
    # ----- sort out the PAST period ---
    nmems = cmip6models.loc[index, "Nmempast"]
    ftype = cmip6models.loc[index, "ftypep"]
    ptype = cmip6models.loc[index, "ptype"]
    
    for imem in range(1,nmems+1,1):
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))

        print("Outputting past for "+modname+" "+memstr)
            
            
        # check if a special order is needed
        changeorder = cmip6models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            if (type(changeorder) == str):
                changeordernp = np.array(changeorder.split(","))
                if (math.isnan(ptype)):
                    memstr = "r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
                else:
                    memstr = "r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))
            
        histdir = glob(histpath+"tas/"+modname+"/"+memstr+"/*/")
        histdir = histdir[0]


        # read in zonal tas, concatenate historical to 2005 and rcp8.5 to 2014
        tas=read.read_sfc(histdir+"*.nc", datebegp, dateendp)
        
        # check the array length.  
        if (tas.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but tas has size "+str(tas.time.size))
            
        # calculate the JJA mean
        tasdjf = cal.season_mean(tas,"tas", season="DJF")
        tasgm = avg.cosweightlonlat(tasdjf,0,360,-90,90)

        # save the first member
        if (imem == 1):
            tas1memp[index]=tasgm

        # calculate the ensemble mean
        tasemp[index]=tasemp[index]+tasgm/float(nmems)
        
    # ---sort out the future period
    nmems = cmip6models.loc[index, "Nmemfuture"]
    ftype = cmip6models.loc[index, "ftypef"]
    
    for imem in range(1,nmems+1,1):
        
        if (math.isnan(ptype)):
            memstr="r"+str(imem)+"i1p1f"+str(int(ftype))
        else:
            memstr="r"+str(imem)+"i1p"+str(int(ptype))+"f"+str(int(ftype))
        
        # check if a special order is needed
        changeorder = cmip6models.loc[index, "specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            if (type(changeorder) == str):
                changeordernp = np.array(changeorder.split(","))
                if (math.isnan(ptype)):
                    memstr = "r"+str(changeordernp[imem-1])+"i1p1f"+str(int(ftype))
                else:
                    memstr = "r"+str(changeordernp[imem-1])+"i1p"+str(int(ptype))+"f"+str(int(ftype))

        print("Outputting future for "+modname+" "+memstr)
                    
        ssp585dir = glob(ssp585path+"/tas/"+modname+"/"+memstr+"/*/")
        ssp585dir = ssp585dir[0]
        
        tas = read.read_sfc(ssp585dir+"*.nc",datebegf, dateendf)
        
        if (tas.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but tas has size "+str(tas.time.size))
            
        tasdjf = cal.season_mean(tas,"tas", season="DJF")
        tasgm = avg.cosweightlonlat(tasdjf, 0, 360, -90, 90)
        
        # save the first member
        if (imem == 1):
            tas1memf[index]=tasgm
            
        # calculate the ensemble mean
        tasemf[index]=tasemf[index]+tasgm/float(nmems)    

Outputting past for ACCESS-CM2 r1i1p1f1
Outputting past for ACCESS-CM2 r2i1p1f1
Outputting future for ACCESS-CM2 r1i1p1f1
Outputting past for ACCESS-ESM1-5 r1i1p1f1
Outputting past for ACCESS-ESM1-5 r2i1p1f1
Outputting past for ACCESS-ESM1-5 r3i1p1f1
Outputting future for ACCESS-ESM1-5 r1i1p1f1
Outputting future for ACCESS-ESM1-5 r2i1p1f1
Outputting future for ACCESS-ESM1-5 r3i1p1f1
Outputting past for AWI-CM-1-1-MR r1i1p1f1
Outputting past for AWI-CM-1-1-MR r2i1p1f1
Outputting past for AWI-CM-1-1-MR r3i1p1f1
Outputting past for AWI-CM-1-1-MR r4i1p1f1
Outputting past for AWI-CM-1-1-MR r5i1p1f1
Outputting future for AWI-CM-1-1-MR r1i1p1f1
Outputting past for BCC-CSM2-MR r1i1p1f1
Outputting past for BCC-CSM2-MR r2i1p1f1
Outputting past for BCC-CSM2-MR r3i1p1f1
Outputting future for BCC-CSM2-MR r1i1p1f1
Outputting past for CAMS-CSM1-0 r1i1p1f1
Outputting future for CAMS-CSM1-0 r1i1p1f1
Outputting future for CAMS-CSM1-0 r2i1p1f1
Outputting past for CanESM5 r1i1p1f1
Outputting past for CanE

In [5]:
tas1memp.to_netcdf(path=pathout+"globalmeantascmip6.nc")
tas1memf.to_netcdf(path=pathout+"globalmeantascmip6.nc", mode="a")
tasemp.to_netcdf(path=pathout+"globalmeantascmip6.nc", mode="a")
tasemf.to_netcdf(path=pathout+"globalmeantascmip6.nc", mode="a")